In [3]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


userId — идентификаторы пользователей, которые выставляли оценки фильмам;

movieId — идентификатор фильма.

rating — выставленный рейтинг фильма; 

timestamp — время выставления рейтинга. Это популярный формат даты и времени unixtime. Показывает количество секунд, прошедшее с 1 января 1970 года. Вы будете очень часто встречать этот формат.

In [4]:
len(ratings['userId'].unique())

671

In [5]:
min(ratings['rating'])

0.5

In [6]:
ratings.groupby('userId').count().head() #группировка по полю, считая суммарное количество других полей, без значений

,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100


In [7]:
ratings_count = ratings.groupby('userId').count().reset_index()[['userId', 'movieId']] 
ratings_count.head() #сбросить индексы и взять только нужные столбцы

,userId,movieId
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [8]:
ratings_count.sort_values('movieId',ascending = False).head() 

,userId,movieId
546,547,2391
563,564,1868
623,624,1735
14,15,1700
72,73,1610


In [9]:
film_fans_ratings_count = ratings_count[ ratings_count['movieId'] >= 100 ]

In [10]:
film_fans_ratings_count.sort_values('movieId').head(100)

,userId,movieId
187,188,100
159,160,100
369,370,100
559,560,100
4,5,100
256,257,103
66,67,103
622,623,103
216,217,104
35,36,104


In [11]:
film_fans_user_ids = film_fans_ratings_count['userId'].tolist()

In [12]:
film_fans_user_ids[0:5]

[4, 5, 8, 15, 17]

Осталось отфильтровать исходный датафрейм с рейтингами, оставив только пользователей из списка film_fans_user_ids. Для этого достаточно использовать метод isin.

Метод isin библиотеки Pandas для каждого значения столбца проверяет, входит ли это значение в список film_fans_user_ids. Этот метод удобно использовать в качестве фильтра для датафрейма.

In [13]:
fans_data = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
fans_data.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


После получения датафрейма с киноманами можем посчитать искомую метрику Lifetime. В нашем случае  Lifetime пользователя — это разница между максимальным и минимальным значением столбца timestamp, т. е. средняя разница между последней и первой оценкой. Для расчета вам необходимо выполнить несколько шагов:

1. Сгруппировать датафрейм fans_data по userId и получить минимальное и максимально значение столбца timestamp. Напомним, что это можно получить в одну команду, используя метод agg(['min', 'max']).

2. В отдельном столбце рассчитать разницу 'diff' между минимальным и максимальным значением timestamp. После группировки у вас, скорее всего, будут столбцы со «вложенными» названиями. Для расчета разницы можно использовать следующий синтаксис (датафрейм после группировки и расчета min и max обозначен как min_max_df):

min_max_df['timestamp']['max'] - min_max_df['timestamp']['min']
3. Посчитать среднее значение столбца 'diff'. Это и будет значение Lifetime (в секундах).

4. Переведите это значение в дни.
Ниже показано два решения - мое и преподавателя

In [14]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [15]:
film_fans_ratings_count = ratings_count[ ratings_count['movieId'] >= 100 ]
film_fans_user_ids = film_fans_ratings_count['userId'].tolist()
fans_data = ratings[ ratings['userId'].isin(film_fans_user_ids) ]

In [16]:
min_max_df = fans_data.groupby('userId').timestamp.agg(['min', 'max'])

In [17]:
ratings['diff'] = min_max_df['max'] - min_max_df['min']
ratings['diff']

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4            203560.0
5              2101.0
6                 NaN
7                 NaN
8             85187.0
9                 NaN
10                NaN
11                NaN
12                NaN
13                NaN
14                NaN
15        471393496.0
16                NaN
17             8053.0
18                NaN
19             5282.0
20                NaN
21          1365432.0
22            91491.0
23         18342129.0
24                NaN
25                NaN
26         20267261.0
27                NaN
28                NaN
29                NaN
             ...     
99974             NaN
99975             NaN
99976             NaN
99977             NaN
99978             NaN
99979             NaN
99980             NaN
99981             NaN
99982             NaN
99983             NaN
99984             NaN
99985             NaN
99986             NaN
99987             NaN
99988     

In [18]:
ratings['diff'].mean() / 3600 / 24

455.2285713719898

In [19]:
# Получаем новый датафрейм
film_fans_ratings_count = ratings_count[ ratings_count['movieId'] >= 100 ]
film_fans_user_ids = film_fans_ratings_count['userId'].tolist()
fans_data = ratings[ ratings['userId'].isin(film_fans_user_ids) ]
# Группируем по userId и получаем минимальное и максимальное значения timestamp
min_max_df = fans_data.groupby('userId').agg(['min', 'max'])
# Считаем новый столбец diff
min_max_df['diff'] = min_max_df['timestamp']['max'] - min_max_df['timestamp']['min']
# Получаем ответ в секундах
min_max_df['diff'].mean() / 3600 / 24

min    455.228571
max    455.228571
dtype: float64

In [20]:
import pandas as pd
data = pd.read_csv('ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Если мы хотим получить распределение количества оценок для одного заданного пользователя, то можно было бы просто использовать метод value_counts. Например, для пользователя 1

In [21]:
data[ data['userId'] == 1 ]['rating'].value_counts()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: rating, dtype: int64

Чтобы получить такую таблицу для всех пользователей, давайте в строки поставим значения userId, а в столбцы — значения рейтинга rating. В ячейки поместим количество строк. В итоге получим классическую сводную таблицу. В нашей таблице довольно много неопределенных значений (ячеек с NaN). Довольно логично, т. к. далеко не все пользователи выставляют все значения от 0.5 до 5.0. Чтобы избавиться от них, добавим в сводную таблицу параметр  fill_value. И заменим эти значения на 0:

In [22]:
datasplit = data.pivot_table(index = 'userId', columns = 'rating',values = 'timestamp',aggfunc = 'count',fill_value = 0, margins=True)
datasplit

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0.0,2.0,0.0,7.0,3.0,4.0,1.0,3.0,0.0,0.0,20.0
2,0.0,2.0,0.0,4.0,0.0,36.0,0.0,23.0,0.0,11.0,76.0
3,0.0,0.0,0.0,1.0,3.0,18.0,9.0,11.0,4.0,5.0,51.0
4,0.0,5.0,0.0,5.0,0.0,23.0,0.0,52.0,0.0,119.0,204.0
5,0.0,0.0,1.0,0.0,3.0,3.0,27.0,42.0,19.0,5.0,100.0
6,1.0,2.0,3.0,7.0,0.0,7.0,3.0,11.0,6.0,4.0,44.0
7,0.0,3.0,0.0,5.0,0.0,41.0,0.0,26.0,0.0,13.0,88.0
8,1.0,0.0,0.0,3.0,4.0,13.0,26.0,35.0,16.0,18.0,116.0
9,0.0,0.0,0.0,5.0,0.0,9.0,0.0,23.0,0.0,8.0,45.0


In [23]:
datasplit.sort_values(5.0, ascending=False) #ищем пользователя, который поставил больше всего пятерок

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
All,1101.0,3326.0,1687.0,7271.0,4449.0,20064.0,10538.0,28750.0,7723.0,15095.0,100004.0
564,0.0,152.0,0.0,187.0,0.0,414.0,0.0,707.0,0.0,408.0,1868.0
232,0.0,35.0,0.0,32.0,0.0,96.0,0.0,276.0,0.0,243.0,682.0
242,0.0,0.0,0.0,4.0,0.0,25.0,0.0,151.0,0.0,219.0,399.0
547,53.0,79.0,58.0,204.0,177.0,411.0,378.0,591.0,226.0,214.0,2391.0
30,0.0,20.0,0.0,86.0,0.0,201.0,2.0,505.0,1.0,196.0,1011.0
472,0.0,23.0,1.0,56.0,5.0,188.0,13.0,315.0,40.0,189.0,830.0
102,0.0,8.0,0.0,40.0,0.0,87.0,0.0,369.0,0.0,174.0,678.0
358,0.0,114.0,0.0,91.0,0.0,118.0,0.0,156.0,0.0,138.0,617.0
